In [7]:
import psycopg2
from psycopg2 import sql
#Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="admin",
    password="admin123",
    port=5432
)
cur = conn.cursor()

# Step 1: Drop tables if they exist (safe when re-running)
cur.execute("DROP TABLE IF EXISTS reviews CASCADE;")
cur.execute("DROP TABLE IF EXISTS banks CASCADE;")

# Step 2: Create the schema exactly as required
cur.execute("""
DO $$
BEGIN
    IF NOT EXISTS (SELECT 1 FROM information_schema.schemata WHERE schema_name = 'bank_reviews_schema') THEN
        EXECUTE format('CREATE SCHEMA bank_reviews_schema AUTHORIZATION %I', current_user);
    END IF;
END
$$;
""")

# Ensure we create tables in the new schema (so we don't need privileges on public)
cur.execute("SET search_path TO bank_reviews_schema, public;")

cur.execute("""
CREATE TABLE IF NOT EXISTS banks (
    bank_id SERIAL PRIMARY KEY,
    bank_code VARCHAR(10) UNIQUE NOT NULL,
    bank_name VARCHAR(255) NOT NULL,
    app_name VARCHAR(255)
);
""")



print("Tables banks and reviews created successfully.")

Tables banks and reviews created successfully.


In [ ]:
import pandas as pd
import psycopg2
from psycopg2 import extras # <-- New import
import ast # For safe string evaluation

# --- Table Creation SQL (Same as before) ---
CREATE_TABLES_SQL = """
CREATE TABLE IF NOT EXISTS banks (
    bank_code VARCHAR(50) PRIMARY KEY,
    bank_name VARCHAR(255) NOT NULL,
    app_name VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS reviews (
    review_id VARCHAR(100) PRIMARY KEY,
    bank_code VARCHAR(50) REFERENCES banks (bank_code),
    review_text TEXT,
    rating INTEGER,
    sentiment_label VARCHAR(50),
    sentiment_score NUMERIC(5, 4),
    identified_themes TEXT[] -- PostgreSQL Array type
);
"""

# --- Function to parse themes ---
def parse_themes(themes_str):
    """Converts a string representation of a list into a Python list."""
    if pd.isna(themes_str):
        return []
    try:
        # Use ast.literal_eval for safe conversion of string-list to list
        themes_list = ast.literal_eval(themes_str)
        return [str(t).strip() for t in themes_list if str(t).strip()]
    except Exception:
        # Fallback processing if ast.literal_eval fails
        return [t.strip() for t in themes_str.strip('[]').replace("'", "").split(',') if t.strip()]


conn = None
cur = None

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        host="localhost",
        database="bank_reviews",
        user="admin",
        password="admin123",
        port=5432
    )
    cur = conn.cursor()

    # 1. Create Tables
    print("Ensuring tables 'banks' and 'reviews' exist...")
    cur.execute(CREATE_TABLES_SQL)

    # 2. Load and Prepare Data
    df_banks = pd.read_csv('./data/raw/app_info.csv')
    df_reviews = pd.read_csv('./data/processed/reviews_with_sentiment.csv')

    # Prepare banks data: (bank_code, bank_name, app_name)
    banks_data = df_banks[['bank_code', 'bank_name', 'title']].values.tolist()
    
    # Prepare reviews data: creating list of tuples with processed themes
    reviews_list = []
    for index, row in df_reviews.iterrows():
        reviews_list.append((
            str(row['review_id']),
            str(row['bank_code']),
            str(row['review_text']),
            int(row['rating']),
            str(row['sentiment_label']),
            float(row['sentiment_score']),
            parse_themes(row['identified_themes']) # Processed list
        ))


    # 3. Insert into banks table using execute_batch
    banks_sql = """
    INSERT INTO banks (bank_code, bank_name, app_name)
    VALUES (%s, %s, %s)
    ON CONFLICT (bank_code) DO NOTHING;
    """
    print(f"Inserting {len(banks_data)} banks...")
    extras.execute_batch(cur, banks_sql, banks_data)
    print("Banks insertion complete.")


    # 4. Insert into reviews table using execute_batch
    reviews_sql = """
    INSERT INTO reviews 
        (review_id, bank_code, review_text, rating, sentiment_label, sentiment_score, identified_themes)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (review_id) DO NOTHING;
    """
    print(f"Inserting {len(reviews_list)} reviews...")
    extras.execute_batch(cur, reviews_sql, reviews_list)
    print("Reviews insertion complete.")

    # 5. Commit changes
    conn.commit()
    print("Data inserted successfully.")

except psycopg2.Error as e:
    print(f"A PostgreSQL error occurred: {e}")
    if conn:
        conn.rollback()
except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    # Close connection and cursor cleanly
    if cur:
        cur.close()
    if conn:
        conn.close()
        print("Database connection closed.")

Ensuring tables 'banks' and 'reviews' exist...
Inserting 3 banks...
Banks insertion complete.
Inserting 1200 reviews...
Reviews insertion complete.
Data inserted successfully.
Database connection closed.
